To sort the answers in the discussion forum by votes, it appears we have to log in first...
So we use selenium to log in to the site and click on sort by most votes.

Therefore to use this script you need to use your username and password to login.

In [1]:
import os
import time

In [2]:
import requests

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display

In [4]:
from pymongo import MongoClient

In [5]:
HOST_IP = 'localhost'
DB_PORT = 27017
client = MongoClient(HOST_IP, DB_PORT)
db = client.leetcode
collection = db.answers

In [6]:
my_username = "@gmail.com"
my_password = ""

In [7]:
first_select = True

In [8]:
def get_answer_urls(url):
    """ return at most 10 top voted answers
    """
    global first_select
    driver.get(url)
    time.sleep(2)
    dropdown_buttons = driver.find_elements_by_css_selector('button.dropdown-toggle')
    assert len(dropdown_buttons) == 2
    dropdown_buttons[1].click()
    time.sleep(2)
    if first_select:
        driver.find_element_by_css_selector('a.most_votes').click()
        time.sleep(5)
        first_select = False
    elements = driver.find_elements_by_css_selector('ul.topic-list h2.title > a')[:10]
    return [ele.get_attribute("href") for ele in elements]

In [9]:
def get_answer(url):
    """ element.get_attribute('innerHTML')
    elem.get_attribute("outerHTML")
    """
    answer = {}
    driver.get(url)
    time.sleep(2)
    up_button = driver.find_element_by_css_selector('li i.fa-angle-double-up')
    up_button.click()
    driver.implicitly_wait(3)
    answer["url"] = url
    author_element = driver.find_element_by_css_selector("span.username a")
    answer["author"] = author_element.text
    answer["author_url"] = author_element.get_attribute('href')
    vote_element = driver.find_element_by_css_selector("div.vote-count")
    answer["vote"] = int(vote_element.text)
    content_element = driver.find_element_by_css_selector("div.answer .content")
    answer["content"] = content_element.get_attribute('outerHTML')
    return answer

In [10]:
def login(username, password):
    username = driver.find_element_by_id("id_login")
    password = driver.find_element_by_id("id_password")
    username.send_keys(my_username)
    password.send_keys(my_password)
    driver.find_element_by_css_selector('button.btn-primary').click()

In [14]:
#chrome_driver = "~/hub/personal/leetcode-best-answers/chromedriver"
chrome_driver = "~/Awesome/github/leetcode-best-answers/chromedriver_mac"
os.environ["webdriver.chrome.driver"] = chrome_driver
display = Display(visible=0, size=(1800, 900))
display.start()
driver = webdriver.Chrome(chrome_driver)
driver.set_window_size(1280, 800)
driver.get("https://discuss.leetcode.com/category/8/oj")
time.sleep(2)
driver.find_element_by_link_text("Login").click()
time.sleep(2)

# your username and password
login(username, password)

time.sleep(2)

driver.get("https://discuss.leetcode.com/category/8/oj")

time.sleep(2)
links = [link.get_attribute("href") for link in driver.find_elements_by_css_selector('ul.categories h2 a')]
with open('urls.txt', 'a') as urls_file:
    for index, link in enumerate(links):
        if index < 320:
            continue
        answer_urls = get_answer_urls(link)
        answer_count = len(answer_urls)
        urls_file.write("{}:{}\n".format(str(index), str(answer_count)))
        for answer_url in answer_urls:
            urls_file.write(answer_url + '\n')
        time.sleep(2)

driver.quit()
display.stop()

WebDriverException: Message: 'chromedriver_mac' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
"""
# could be useful in the future
cookies = driver.get_cookies()
headers = {
"User-Agent":
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
}
s = requests.session()
s.headers.update(headers)

for cookie in driver.get_cookies():
    c = {cookie['name']: cookie['value']}
    s.cookies.update(c)
"""